In [1]:
from sqlalchemy.orm import sessionmaker
from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories
import pandas as pd
import os
from dotenv import load_dotenv, find_dotenv
import time
import numpy as np
import re
DEBUG=0

In [2]:
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
import seaborn as sns

In [3]:
# Importing Gensim
import gensim
from gensim import corpora, models
# Compute Perplexity
from gensim.models import CoherenceModel
np.random.seed(42)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
import pyLDAvis.gensim
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
pyLDAvis.enable_notebook()

In [5]:
#from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.model_selection import GridSearchCV
#from sklearn.manifold import TSNE

In [6]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
#nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_md')  # make sure to use larger model!

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string
#!python -m spacy download en_core_web_lg

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\site-packages\thinc\check.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import defaultdict, Sequence, Sized, Iterable, Callable


In [7]:
# run this from a normal command line
#python -m spacy download en_core_web_md

In [8]:
db=connect_db() #establish connection / creates database on first run
Session = sessionmaker(bind=db)
session = Session()

In [9]:
df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
df.sample(5)


,poi_id,name,build_year,build_decade,build_year_clean,demolished_year,address,latitude,longitude,external_url,image_url,heritage_status,current_use,poi_type,poi_type_simple,source,details
10,7615,ONE HUNDRED AND TWENTY MIRRORS,2016,2010.0,2016.0,None,223 Gladys Allison Pl Toronto ON M2N 4T5,43.768488,-79.409820,None,None,None,None,"Public Art, STEEL, WOOD, AUDIO COMPONENTS",Art,https://www.toronto.ca/city-government/data-re...,"Public Art. \r\n Media: STEEL, WOOD, AUDIO CO..."
1222,2131,West Toronto Junction,None,NaN,NaN,None,,43.672444,-79.467845,http://torontoplaques.com/Pages/West_Toronto_J...,None,None,None,Plaque,Plaque,http://torontoplaques.com,Attached to this eastbound St. Clair Avenue We...
93,131,Lewis Lukes House,1887,1880.0,1887.0,None,"37 Madison Avenue The Annex Toronto, ON",43.668780,-79.403050,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,"West Annex Phase I, Madison Avenue, Heritage C...",None,Detached house,Building,http://www.acotoronto.ca/,This property contributes to the Toronto Annex...
4751,7816,MEMORY BANKS,1998,1990.0,1998.0,None,460 Crawford St Toronto ON M6G 3J6,43.656712,-79.420070,None,None,None,None,"Public Art, MASONRY",Art,https://www.toronto.ca/city-government/data-re...,Public Art. \r\n Media: MASONRY\r\n LOCATION:...
788,1382,Glenmaple,1993,1990.0,1993.0,None,"1319 Neilson Road Malvern Scarborough, ON",43.808512,-79.218207,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,No heritage status,None,High-rise,Building,http://www.acotoronto.ca/,None


Create dataframe that only contains Toronto Plaques info 

In [10]:
df_plaques = df[( df['source'].str.contains('torontoplaques')) ]

In [11]:
df_plaques.head()

,poi_id,name,build_year,build_decade,build_year_clean,demolished_year,address,latitude,longitude,external_url,image_url,heritage_status,current_use,poi_type,poi_type_simple,source,details
148,325,The Beatles,None,NaN,NaN,None,,43.662050,-79.379730,http://torontoplaques.com/Pages/Beatles.html,None,None,None,Plaque,Plaque,http://torontoplaques.com,"Inside the former Maple Leaf Gardens, now a Lo..."
198,324,Alexander Muir 1830-1906,None,NaN,NaN,None,,43.663153,-79.327211,http://torontoplaques.com/Pages/Alexander_Muir...,None,None,None,Plaque,Plaque,http://torontoplaques.com,A maple tree on the southwest corner of Laing ...
199,326,Bernard Keble Sandwell (1876-1954),None,NaN,NaN,None,,43.650811,-79.382486,http://torontoplaques.com/Pages/Bernard_Keble_...,None,None,None,Plaque,Plaque,http://torontoplaques.com,I used to read Saturday Night magazine quite r...
201,329,Boris Volkoff,None,NaN,NaN,None,,43.671150,-79.387080,http://torontoplaques.com/Pages/Boris_Volkoff....,None,None,None,Plaque,Plaque,http://torontoplaques.com,This 2009 Heritage Toronto plaque can be found...
202,330,Bruce Mackey,None,NaN,NaN,None,,43.661200,-79.345950,http://torontoplaques.com/Pages/Bruce_Mackey_D...,None,None,None,Plaque,Plaque,http://torontoplaques.com,"In Bruce Mackey Park, west of the railway trac..."


## Let's play around with Spacy capabilities first

In [32]:
# let's examine the details for one plaque
mytext = df_plaques.iloc[0,16]
mytext

'Inside the former Maple Leaf Gardens, now a Loblaws, at the "Canteen" on the main level imbedded in the top of a table is this plaque. Here\'s what it says: Beatlemania wasn\'t quite dead, but it was clearly on the wane. In 1964, on the first North American tour, the Fab Four were greeted by 10,000 fans at the airport in Toronto, and were mobbed at their hotel - where one enterprising teenaged girl even managed to hide in their closet. Their two shows set an attendance record at Maple Leaf Gardens. A year later, when they rolled into town on the heels of their wild and triumphant concert at Shea Stadium, Gardens\' owner Harold Ballard turned up the heat, turned off the water fountains, and made a fortune selling soft drinks as the lads from Liverpool again packed the arena for two shows. But by August 17, 1966, something had changed. John Lennon\'s crack about the band being more popular than Jesus had stirred controversy and protest. Only 800 fans were at the airport to greet the Bea

In [13]:
mytokens = nlp(mytext)

# examine what spacy thinks of each word
tokens_lst = []
for token in mytokens:
   # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
    mytoken = {}
    mytoken['1_text'] = token.text
    mytoken['lemma']=token.lemma_
    mytoken['POS'] =  token.pos_
    mytoken['tag'] = token.tag_
    mytoken['dep'] = token.dep_
    mytoken['shape'] = token.shape_
    mytoken['Is_Alpha?'] = token.is_alpha
    mytoken['Is_stop?'] =  token.is_stop
    tokens_lst.append(mytoken)
tokens_df = pd.DataFrame(tokens_lst)
    

In [14]:
tokens_df.iloc[37:51, :]

,1_text,Is_Alpha?,Is_stop?,POS,dep,lemma,shape,tag
37,Beatlemania,True,False,PROPN,nsubj,beatlemania,Xxxxx,NNP
38,was,True,False,VERB,ROOT,be,xxx,VBD
39,n't,False,False,ADV,neg,not,x'x,RB
40,quite,True,False,ADV,advmod,quite,xxxx,RB
41,dead,True,False,ADJ,acomp,dead,xxxx,JJ
42,",",False,False,PUNCT,punct,",",",",","
43,but,True,False,CCONJ,cc,but,xxx,CC
44,it,True,False,PRON,nsubj,-PRON-,xx,PRP
45,was,True,False,VERB,conj,be,xxx,VBD
46,clearly,True,False,ADV,advmod,clearly,xxxx,RB


In [15]:
# you can use explain to look up any labels you're not familiar with
spacy.explain("JJ")

'adjective'

#### Now let's look at Named Entities in this plaque description

In [16]:
ner_lst=[]
for ent in mytokens.ents:
   # print(ent.text, ent.start_char, ent.end_char, ent.label_)
    my_ners = {}
    my_ners['1_NER'] = ent.text
    my_ners['2_Label']=ent.label_
    ner_lst.append(my_ners)
ners_df = pd.DataFrame(ner_lst)

In [17]:
ners_df.head(20)

,1_NER,2_Label
0,Maple Leaf Gardens,FAC
1,Loblaws,GPE
2,"the ""Canteen""",FAC
3,Beatlemania,GPE
4,1964,DATE
5,first,ORDINAL
6,North American,NORP
7,the Fab Four,PRODUCT
8,"10,000",CARDINAL
9,Toronto,GPE


In [18]:
# count of NERs in plaque
ners_df['1_NER'].value_counts()

Beatles                4
Maple Leaf Gardens     3
two                    2
North American         2
1966                   1
1964                   1
the day                1
Liverpool              1
the Fab Four           1
35                     1
Yesterday              1
Harold Ballard         1
10,000                 1
Loblaws                1
Stephen Brunt          1
John Lennon            1
Shea Stadium           1
the "Canteen"          1
Long Tall Sally        1
Eleanor Rigby          1
Gardens                1
only played a dozen    1
all three              1
the Toronto Star       1
Arthur Zeldin          1
Only 800               1
A year later           1
Toronto                1
John Lennon's          1
only eight             1
Zeldin                 1
Paul McCartney         1
first                  1
The Ronettes           1
Jesus                  1
August 17, 1966        1
Beatlemania            1
Name: 1_NER, dtype: int64

In [19]:
displacy.render(nlp(mytext), jupyter=True, style='ent')

#### Sentences

In [20]:
# look at sentences
for sent in mytokens.sents:
    print(sent)

Inside the former Maple Leaf Gardens, now a Loblaws, at the "Canteen" on the main level imbedded in the top of a table is this plaque.
Here's what it says: Beatlemania wasn't quite dead, but it was clearly on the wane.
In 1964, on the first North American tour, the Fab Four were greeted by 10,000 fans at the airport in Toronto, and were mobbed at their hotel - where one enterprising teenaged girl even managed to hide in their closet.
Their two shows set an attendance record at Maple Leaf Gardens.
A year later, when they rolled into town on the heels of their wild and triumphant concert at Shea Stadium, Gardens' owner Harold Ballard turned up the heat, turned off the water fountains, and made a fortune selling soft drinks as the lads from Liverpool again packed the arena for two shows.
But by August 17, 1966, something had changed.
John Lennon's crack about the band being more popular than Jesus had stirred controversy and protest.
Only 800 fans were at the airport to greet the Beatles,

#### Check similarity between words

In [21]:
# Similarity
tokens = nlp(u'tree car truck')
for token1 in tokens:
    for token2 in tokens:
        if token1 != token2:
            print(token1.text, token2.text, token1.similarity(token2))

tree car 0.24681789
tree truck 0.2788549
car tree 0.24681789
car truck 0.7113439
truck tree 0.2788549
truck car 0.7113439


### Create a tokenizer for our plaque details

Create function to tokenize text in the Details columns
* use Spacy English() parser
* run through NLP pipeline
* convert words to lowercase and lemmatize (stem) (add special handling for Pronouns Spacy lemmatizes as "-PRON-")
* remove stopwords and punctuation
* return string

In [22]:

punctuations = string.punctuation
stopwords = list(STOP_WORDS)
def spacy_tokenizer(plaque_descr):
    if plaque_descr != None:
        mytokens = nlp(plaque_descr)
        mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
        mytokens = [ re.sub('\\d+', '', word) for word in mytokens if word not in stopwords and word not in punctuations ]
        mytokens = " ".join([i for i in mytokens])
        return mytokens
    else:
        return ""

In [23]:
# example processing of a sentence
spacy_tokenizer("Writing in the Toronto 19 Star, Arthur Zeldin had it right 2000")

'write toronto  star arthur zeldin right '

In [24]:
df_plaques['details_parsed'] = df_plaques['details'].apply(lambda x: spacy_tokenizer(x))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_plaques.head()

,poi_id,name,build_year,build_decade,build_year_clean,demolished_year,address,latitude,longitude,external_url,image_url,heritage_status,current_use,poi_type,poi_type_simple,source,details,details_parsed
148,325,The Beatles,None,NaN,NaN,None,,43.662050,-79.379730,http://torontoplaques.com/Pages/Beatles.html,None,None,None,Plaque,Plaque,http://torontoplaques.com,"Inside the former Maple Leaf Gardens, now a Lo...",inside maple leaf gardens loblaws canteen main...
198,324,Alexander Muir 1830-1906,None,NaN,NaN,None,,43.663153,-79.327211,http://torontoplaques.com/Pages/Alexander_Muir...,None,None,None,Plaque,Plaque,http://torontoplaques.com,A maple tree on the southwest corner of Laing ...,maple tree southwest corner laing street memor...
199,326,Bernard Keble Sandwell (1876-1954),None,NaN,NaN,None,,43.650811,-79.382486,http://torontoplaques.com/Pages/Bernard_Keble_...,None,None,None,Plaque,Plaque,http://torontoplaques.com,I used to read Saturday Night magazine quite r...,use read saturday night magazine regularly die...
201,329,Boris Volkoff,None,NaN,NaN,None,,43.671150,-79.387080,http://torontoplaques.com/Pages/Boris_Volkoff....,None,None,None,Plaque,Plaque,http://torontoplaques.com,This 2009 Heritage Toronto plaque can be found...,heritage toronto plaque find wall building ea...
202,330,Bruce Mackey,None,NaN,NaN,None,,43.661200,-79.345950,http://torontoplaques.com/Pages/Bruce_Mackey_D...,None,None,None,Plaque,Plaque,http://torontoplaques.com,"In Bruce Mackey Park, west of the railway trac...",bruce mackey park west railway track east de g...


# Vectorize using CountVectorizer 
* select unigrams and bigrams
* max_df: ignore words that have a document frequency > 0.8 to try to weed out very common words
* remove numbers

In [26]:
# Creating a vectorizer
vectorizer = CountVectorizer(min_df=5, ngram_range=(1,2), max_df=0.8, stop_words='english', lowercase=True, max_features=2000) 
detail_vec = vectorizer.fit_transform(df_plaques['details_parsed'])

df_vect = pd.DataFrame(detail_vec.toarray(), columns=vectorizer.get_feature_names())
df_vect.head()

,abandon,able,abolitionist,aboriginal,abraham,academy,access,acclaim,accommodate,accompany,...,york plaque,york street,york toronto,york township,yorkville,yorkville avenue,young,youth,étienne,étienne brûlé
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [27]:

plaques = [df_vect.columns[df_vect.loc[index,:].nonzero()] for index in df_vect.index]

In [28]:
plaques[1]

Index(['alexander', 'america', 'away', 'block', 'british', 'canada', 'corner',
       'fall', 'follow', 'forever', 'grand', 'idea', 'inspire', 'lane', 'leaf',
       'like', 'lodge', 'longer', 'maple', 'maple leaf', 'memory', 'national',
       'near', 'nearby', 'page', 'plaque near', 'popular', 'principal',
       'print', 'public', 'public school', 'queen', 'queen street', 'read',
       'school', 'second', 'song', 'south', 'south queen', 'southwest',
       'southwest corner', 'street', 'tree', 'word', 'write', 'year'],
      dtype='object')

# Use Gensim LDA

* Dictionary:  Gensim Dictionary creates a mapping between words and their integer IDs for faster lookups
* Corpus: The corpus is the frequency of each word within each document.  Converts list of documents (corpus) into Document Term Matrix using dictionary prepared above.


In [ ]:
dictionary = corpora.Dictionary(plaques)

In [ ]:

corpus = [dictionary.doc2bow(plaque) for plaque in plaques]


* With LDA, we don't know in advance how many topics there are in the corpus
* we can make a guess, or we can loop through a range of possible topic values and record the Coherence scores for evaluation to try to find the best number of topics

In [ ]:
Lda = models.LdaMulticore

In [ ]:
# source: https://github.com/smsubrahmannian/Topic-Modeling/blob/master/code/Preparing%20a%20Topic%20model.ipynb
Lda = models.LdaMulticore
coherenceList_umass = []
coherenceList_cv = []
#num_topics_list = np.arange(3,20)
for num_topics in range(7,13):
    lda= Lda(corpus, num_topics=num_topics,id2word = dictionary, 
             passes=20,chunksize=4000,random_state=42)
    cm = CoherenceModel(model=lda, corpus=corpus, 
                        dictionary=dictionary, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    cm_cv = CoherenceModel(model=lda, corpus=corpus,
                           texts=plaques, dictionary=dictionary, coherence='c_v')
    coherenceList_cv.append(cm_cv.get_coherence())
    vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
    pyLDAvis.save_html(vis,f'pyLDAvis_{num_topics}.html')

In [ ]:
#https://github.com/smsubrahmannian/Topic-Modeling/blob/master/code/Preparing%20a%20Topic%20model.ipynb
num_topics_list = np.arange(5,20)
plotData = pd.DataFrame({'Number of topics':num_topics_list,
                         'CoherenceScore':coherenceList_umass})
f,ax = plt.subplots(figsize=(10,6))
sns.set_style("darkgrid")
sns.pointplot(x='Number of topics',y= 'CoherenceScore',data=plotData)
plt.axhline(y=-1.363562)
plt.title('Topic coherence (Umass)')
plt.savefig('Topic coherence plot.png')

In [ ]:
plotData = pd.DataFrame({'Number of topics':num_topics_list,
                         'CoherenceScore':coherenceList_cv})
f,ax = plt.subplots(figsize=(10,6))
sns.set_style("darkgrid")
sns.pointplot(x='Number of topics',y= 'CoherenceScore',data=plotData)
plt.axhline(y=0.484941)
plt.title('Topic coherence CV')
plt.savefig('Topic coherence plot.png')

In [ ]:
coherenceList_umass

In [ ]:
coherenceList_cv

#### persist our selected model so we can evaluate it in another notebook (saves memory!)

In [ ]:
Lda = models.LdaMulticore
lda_final= Lda(corpus, num_topics=9,id2word = dictionary, passes=20,chunksize=4000,random_state=43)

In [ ]:

lda_final.save('lda_final_9')
dictionary.save('dictionary')
corpora.MmCorpus.serialize('corpus_9.mm', corpus)

In [ ]:
num_topics =9
vis = pyLDAvis.gensim.prepare(lda_final, corpus, dictionary,sort_topics=False)
pyLDAvis.save_html(vis,f'pyLDAvis_{num_topics}.html')
vis

### What is the dominant topic for each plaque
* source: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
* for all 1029 plaques, list the dominant topic

In [ ]:
# source: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
def format_topics_sentences(ldamodel=lda_final, corpus=corpus, texts=plaques):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # prints the topics for each document and their percent contribution
        # i.e. plaque 1 is 95% topic 1 and 4% topic 8 [(1, 0.9483137), (8, 0.043318484)]
        # plaque2 is 39% topic 1, 37% topic 2 and 22% topic 3 [(1, 0.3942705), (2, 0.36660466), (3, 0.22493163)]
        
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_final, corpus=corpus, texts=plaques)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Plaque_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

* according to this, the first plaque (our text about the Beatles) corresponds to "topic 2" in pyLDAvis, to which we assigned the subject "Maple Leaf / Hockey"
* this isn't bad, since it is related to Maple Leaf gardens, however, it has completely missed what I woudl consider the dominant "music" theme of the Beatles!!

### Most representative plaque for each topic

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet

In [ ]:
print(sent_topics_sorteddf_mallet.iloc[0,3])